In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Electricity Production Data

## Import data

In [2]:
df = pd.read_csv('data/Electric_Production.csv')
data = np.array(df['value'])
dataset = np.reshape(data,(-1,1)).astype('float32')
len(dataset)

397

## Data preprocessing

In [3]:
#Create dataset from raw data
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
	    a = dataset[i:(i+look_back), 0]
	    dataX.append(a)
	    dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

def create_model(n_units = 100, window_size = 10, af = "relu"):
    model = Sequential()
    model.add(LSTM(n_units, input_dim=window_size, activation=af))
    model.add(Dense(1))
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model

In [4]:
#Data point from test_index_from is the starting index of the testing set
test_index_from = int(len(dataset) * 0.75)
train = dataset[0:test_index_from,:]

In [5]:
#Parameter tuning function
def parameter_tuning(trainset, window_sizes = [], list_n_units = [], activation_funcs = []):
    best_val_loss = 1e1000000000
    best_window_size = 0
    best_n_units = 0
    best_af = " "
    for window_size in window_sizes:
        trainX, trainY = create_dataset(trainset, window_size)
        trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
        for n_units in list_n_units:
            for af in activation_funcs:
                lstm = create_model(n_units = n_units, window_size = window_size, af = af)
                history = lstm.fit(trainX, trainY, epochs=120, batch_size=32, verbose=0,validation_split=0.3)
                val_loss = history.history['val_loss']
                if val_loss[-1] < best_val_loss:
                    best_val_loss = val_loss[-1]
                    best_window_size = window_size
                    best_n_units = n_units
                    best_af = af
    print("Best parameter set: window_size: " + str(best_window_size) + " n_units: " + str(best_n_units) + " activation function: " + best_af)
            

In [6]:
window_sizes = [10, 15, 20, 25]
list_n_units = [20, 40, 60, 80, 100]
activation_funcs = ["relu"]

parameter_tuning(dataset, window_sizes=window_sizes, list_n_units=list_n_units, activation_funcs = activation_funcs)

Best parameter set: window_size: 15 n_units: 60 activation function: relu


# Microsoft Stock Data Price

## Import data

In [8]:
df = pd.read_csv('data/Microsoft_Stock.csv')
data = np.array(df['Close'])
dataset = np.reshape(data,(-1,1)).astype('float32')
len(dataset)

1511

In [9]:
test_index_from = int(len(dataset) * 0.75)
train = dataset[0:test_index_from,:]

In [10]:
window_sizes = [40, 60, 80, 100]
list_n_units = [40, 60, 80, 100]
activation_funcs = ["relu"]

parameter_tuning(dataset, window_sizes=window_sizes, list_n_units=list_n_units, activation_funcs = activation_funcs)

Best parameter set: window_size: 40 n_units: 100 activation function: relu


# Microsoft Stock Data Volume

In [11]:
data = np.array(df['Volume'])
dataset = np.reshape(data,(-1,1)).astype('float32')
len(dataset)

1511

In [12]:
test_index_from = int(len(dataset) * 0.75)
train = dataset[0:test_index_from,:]

In [13]:
window_sizes = [40, 60, 80, 100]
list_n_units = [40, 60, 80, 100]
activation_funcs = ["relu"]

parameter_tuning(dataset, window_sizes=window_sizes, list_n_units=list_n_units, activation_funcs = activation_funcs)

Best parameter set: window_size: 40 n_units: 80 activation function: relu
